In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "centaur/fty/top" :dir :system)
(include-book "std/util/define" :dir :system)
(include-book "llm-types")        ; chat-message-list-p, model-info-p
(include-book "context-manager")  ; add-message, truncate-to-fit, etc.


Summary
Form:  ( INCLUDE-BOOK "centaur/fty/top" ...)
Rules: NIL
Time:  0.40 seconds (prove: 0.00, print: 0.00, other: 0.40)
 "/home/acl2/books/centaur/fty/top.lisp"

The event ( INCLUDE-BOOK "std/util/define" ...) is redundant.  See
:DOC redundant-events.

Summary
Form:  ( INCLUDE-BOOK "std/util/define" ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT

Summary
Form:  ( INCLUDE-BOOK "llm-types" ...)
Rules: NIL
Time:  0.04 seconds (prove: 0.00, print: 0.00, other: 0.04)
 "/workspaces/verified-agent/src/llm-types.lisp"
Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK "context-manager" ...)
Rules: NIL
Time:  0.27 seconds (prove: 0.00, print: 0.00, other: 0.27)
 "/workspaces/verified-agent/src/context-manager.lisp"


In [3]:
;; Structured error kinds for rich error handling
(fty::deftagsum error-kind
  (:none ())
  (:resource-exhausted ())
  (:precondition-failed ((reason stringp)))
  (:tool-error ((tool-name symbolp) (message stringp)))
  (:max-iterations ())
  :layout :list)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [4]:
;; Tool specification - describes a tool's requirements
(fty::defprod tool-spec
  ((name symbolp)
   (required-access natp :default 0)      ; 0=none, 1=read, 2=read-write
   (requires-execute booleanp :default nil)
   (token-cost natp :default 0)
   (time-cost natp :default 0))
  :layout :list)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [5]:
;; Core agent state - tracks resources, permissions, conversation, and status
(fty::defprod agent-state
  (;; Step and resource management
   (step-counter natp :default 0)
   (max-steps natp :default 100)
   (token-budget natp :default 10000)
   (time-budget natp :default 3600)
   ;; Permissions
   (file-access natp :default 0)          ; granted access level (0/1/2)
   (execute-allowed booleanp :default nil)
   ;; Conversation history (for LLM context)
   (messages chat-message-list-p :default nil)
   (max-context-tokens natp :default 8000) ; from model-info->loaded-context-length
   (system-prompt stringp :default "")     ; cached for reference
   ;; Completion status
   (satisfaction natp :default 0)         ; 0-100 scale
   (done booleanp :default nil)
   (error-state error-kind-p :default '(:none)))
  :layout :list)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [6]:
;; Actions the agent can take
(fty::deftagsum agent-action
  (:tool-call ((tool-name symbolp) (query stringp)))
  (:final-answer ((answer stringp)))
  (:no-action ())
  :layout :list)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [7]:
(defconst *satisfaction-threshold* 100)


Summary
Form:  ( DEFCONST *SATISFACTION-THRESHOLD* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SATISFACTION-THRESHOLD*


In [8]:
;; Access level constants
(defconst *access-none* 0)
(defconst *access-read* 1)
(defconst *access-read-write* 2)


Summary
Form:  ( DEFCONST *ACCESS-NONE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-NONE*

Summary
Form:  ( DEFCONST *ACCESS-READ* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-READ*

Summary
Form:  ( DEFCONST *ACCESS-READ-WRITE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-READ-WRITE*


In [9]:
;; Check if granted access is sufficient for required access
(define access-sufficient-p ((required natp) (granted natp))
  :returns (result booleanp)
  (<= required granted))


Since ACCESS-SUFFICIENT-P is non-recursive, its admission is trivial.
We observe that the type of ACCESS-SUFFICIENT-P is described by the
theorem 
(OR (EQUAL (ACCESS-SUFFICIENT-P REQUIRED GRANTED)
           T)
    (EQUAL (ACCESS-SUFFICIENT-P REQUIRED GRANTED)
           NIL)).

Computing the guard conjecture for ACCESS-SUFFICIENT-P....

The guard conjecture for ACCESS-SUFFICIENT-P is trivial to prove, given
the :compound-recognizer rule NATP-COMPOUND-RECOGNIZER.  ACCESS-SUFFICIENT-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN ACCESS-SUFFICIENT-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-ACCESS-SUFFICIENT-P
          (B* ((RESULT (ACCESS-SUFFICIENT-P REQUIRED GRANTED)))
            (BOOLEANP RESULT))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'ACCESS-SUFFICIENT-P
                                                        ID WORLD))
          :RULE-CLASSES 

In [10]:
;; Check if tool permissions are satisfied
(define tool-permitted-p ((tool tool-spec-p) (st agent-state-p))
  :returns (result booleanp)
  (b* ((required-access (tool-spec->required-access tool))
       (requires-exec (tool-spec->requires-execute tool))
       (granted-access (agent-state->file-access st))
       (exec-allowed (agent-state->execute-allowed st)))
    (and (access-sufficient-p required-access granted-access)
         (or (not requires-exec) exec-allowed))))


Since TOOL-PERMITTED-P is non-recursive, its admission is trivial.
We observe that the type of TOOL-PERMITTED-P is described by the theorem
(OR (EQUAL (TOOL-PERMITTED-P TOOL ST) T)
    (EQUAL (TOOL-PERMITTED-P TOOL ST) NIL)).
We used the :type-prescription rule AGENT-STATE->EXECUTE-ALLOWED$INLINE.

Computing the guard conjecture for TOOL-PERMITTED-P....

The non-trivial part of the guard conjecture for TOOL-PERMITTED-P is

Goal
(IMPLIES (AND (AGENT-STATE-P ST)
              (TOOL-SPEC-P TOOL))
         (AND (TOOL-SPEC-P TOOL)
              (LET ((REQUIRED-ACCESS (TOOL-SPEC->REQUIRED-ACCESS TOOL)))
                (AND (TOOL-SPEC-P TOOL)
                     (AGENT-STATE-P ST)
                     (LET ((GRANTED-ACCESS (AGENT-STATE->FILE-ACCESS ST)))
                       (AND (AGENT-STATE-P ST)
                            (NATP REQUIRED-ACCESS)
                            (NATP GRANTED-ACCESS))))))).

Q.E.D.

That completes the proof of the guard theorem for TOOL-PERMITTED-P.
TOOL-PE

In [11]:
;; Check if budget is sufficient for tool
(define tool-budget-sufficient-p ((tool tool-spec-p) (st agent-state-p))
  :returns (result booleanp)
  (b* ((token-cost (tool-spec->token-cost tool))
       (time-cost (tool-spec->time-cost tool))
       (token-budget (agent-state->token-budget st))
       (time-budget (agent-state->time-budget st)))
    (and (<= token-cost token-budget)
         (<= time-cost time-budget))))


Since TOOL-BUDGET-SUFFICIENT-P is non-recursive, its admission is trivial.
We observe that the type of TOOL-BUDGET-SUFFICIENT-P is described by
the theorem 
(OR (EQUAL (TOOL-BUDGET-SUFFICIENT-P TOOL ST)
           T)
    (EQUAL (TOOL-BUDGET-SUFFICIENT-P TOOL ST)
           NIL)).

Computing the guard conjecture for TOOL-BUDGET-SUFFICIENT-P....

The non-trivial part of the guard conjecture for TOOL-BUDGET-SUFFICIENT-P
is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (TOOL-SPEC-P TOOL))
 (AND (TOOL-SPEC-P TOOL)
      (LET ((TOKEN-COST (TOOL-SPEC->TOKEN-COST TOOL)))
        (AND (TOOL-SPEC-P TOOL)
             (LET ((TIME-COST (TOOL-SPEC->TIME-COST TOOL)))
               (AND (AGENT-STATE-P ST)
                    (LET ((TOKEN-BUDGET (AGENT-STATE->TOKEN-BUDGET ST)))
                      (AND (AGENT-STATE-P ST)
                           (LET ((TIME-BUDGET (AGENT-STATE->TIME-BUDGET ST)))
                             (AND (RATIONALP TOKEN-BUDGET)
                                  (RATIONA

In [12]:
;; Combined check: can we invoke this tool?
(define can-invoke-tool-p ((tool tool-spec-p) (st agent-state-p))
  :returns (result booleanp)
  (and (tool-permitted-p tool st)
       (tool-budget-sufficient-p tool st)))


Since CAN-INVOKE-TOOL-P is non-recursive, its admission is trivial.
We observe that the type of CAN-INVOKE-TOOL-P is described by the theorem
(OR (EQUAL (CAN-INVOKE-TOOL-P TOOL ST) T)
    (EQUAL (CAN-INVOKE-TOOL-P TOOL ST)
           NIL)).
We used the :type-prescription rule TOOL-BUDGET-SUFFICIENT-P.

Computing the guard conjecture for CAN-INVOKE-TOOL-P....

The guard conjecture for CAN-INVOKE-TOOL-P is trivial to prove, given
the :compound-recognizer rules CONSP-WHEN-AGENT-STATE-P and 
CONSP-WHEN-TOOL-SPEC-P.  CAN-INVOKE-TOOL-P is compliant with Common
Lisp.

Summary
Form:  ( DEFUN CAN-INVOKE-TOOL-P ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:COMPOUND-RECOGNIZER CONSP-WHEN-TOOL-SPEC-P)
        (:TYPE-PRESCRIPTION TOOL-BUDGET-SUFFICIENT-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-CAN-INVOKE-TOOL-P
          (B* ((RESULT (CAN-INVOKE-TOOL-P TOOL ST)))
            (BOOLEANP RESULT))
          :HINTS ((STD::RETU

In [13]:
;; Check if agent has an error
(define has-error-p ((st agent-state-p))
  :returns (result booleanp)
  (not (equal (error-kind-kind (agent-state->error-state st)) :none)))


Since HAS-ERROR-P is non-recursive, its admission is trivial.  We observe
that the type of HAS-ERROR-P is described by the theorem 
(OR (EQUAL (HAS-ERROR-P ST) T) (EQUAL (HAS-ERROR-P ST) NIL)).  

Computing the guard conjecture for HAS-ERROR-P....

The non-trivial part of the guard conjecture for HAS-ERROR-P is

Goal
(IMPLIES (AGENT-STATE-P ST)
         (AND (AGENT-STATE-P ST)
              (ERROR-KIND-P (AGENT-STATE->ERROR-STATE ST)))).

Q.E.D.

That completes the proof of the guard theorem for HAS-ERROR-P.  HAS-ERROR-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN HAS-ERROR-P ...)
Rules: ((:DEFINITION NOT)
        (:REWRITE ERROR-KIND-P-OF-AGENT-STATE->ERROR-STATE))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  18


ACL2 !>>(DEFTHM BOOLEANP-OF-HAS-ERROR-P
          (B* ((RESULT (HAS-ERROR-P ST)))
            (BOOLEANP RESULT))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'HAS-ERROR-P
                                           

In [14]:
;; Check if agent must respond (cannot continue)
(define must-respond-p ((st agent-state-p))
  :returns (result booleanp)
  (or (agent-state->done st)
      (has-error-p st)
      (>= (agent-state->step-counter st)
          (agent-state->max-steps st))
      (= (agent-state->token-budget st) 0)
      (= (agent-state->time-budget st) 0)))


Since MUST-RESPOND-P is non-recursive, its admission is trivial.  We
observe that the type of MUST-RESPOND-P is described by the theorem
(OR (EQUAL (MUST-RESPOND-P ST) T) (EQUAL (MUST-RESPOND-P ST) NIL)).
We used primitive type reasoning and the :type-prescription rules 
AGENT-STATE->DONE$INLINE, AGENT-STATE->TOKEN-BUDGET$INLINE and HAS-ERROR-P.

Computing the guard conjecture for MUST-RESPOND-P....

The guard conjecture for MUST-RESPOND-P is trivial to prove, given
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-
prescription rules AGENT-STATE->MAX-STEPS$INLINE, 
AGENT-STATE->STEP-COUNTER$INLINE, AGENT-STATE->TIME-BUDGET$INLINE and
AGENT-STATE->TOKEN-BUDGET$INLINE.  MUST-RESPOND-P is compliant with
Common Lisp.

Summary
Form:  ( DEFUN MUST-RESPOND-P ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION AGENT-STATE->DONE$INLINE)
        (:TYPE-PRESCRIPTION AGENT-STATE->MAX-STEPS$INLINE)
     

In [15]:
;; Check if agent should continue (has budget and not satisfied)
(define should-continue-p ((st agent-state-p))
  :returns (result booleanp)
  (and (not (must-respond-p st))
       (< (agent-state->satisfaction st) *satisfaction-threshold*)))


Since SHOULD-CONTINUE-P is non-recursive, its admission is trivial.
We observe that the type of SHOULD-CONTINUE-P is described by the theorem
(OR (EQUAL (SHOULD-CONTINUE-P ST) T) (EQUAL (SHOULD-CONTINUE-P ST) NIL)).

Computing the guard conjecture for SHOULD-CONTINUE-P....

The guard conjecture for SHOULD-CONTINUE-P is trivial to prove, given
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-
prescription rule AGENT-STATE->SATISFACTION$INLINE.  SHOULD-CONTINUE-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN SHOULD-CONTINUE-P ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:TYPE-PRESCRIPTION AGENT-STATE->SATISFACTION$INLINE))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-SHOULD-CONTINUE-P
          (B* ((RESULT (SHOULD-CONTINUE-P ST)))
            (BOOLEANP RESULT))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'SHOULD-CONTINUE-P
                                                      

In [16]:
;; Remaining steps (for termination measure)
(define remaining-steps ((st agent-state-p))
  :returns (n natp)
  (nfix (- (agent-state->max-steps st)
           (agent-state->step-counter st))))


Since REMAINING-STEPS is non-recursive, its admission is trivial. 
We observe that the type of REMAINING-STEPS is described by the theorem
(AND (INTEGERP (REMAINING-STEPS ST)) (<= 0 (REMAINING-STEPS ST))).
We used the :type-prescription rule NFIX.

Computing the guard conjecture for REMAINING-STEPS....

The guard conjecture for REMAINING-STEPS is trivial to prove, given
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P, primitive type
reasoning and the :type-prescription rules AGENT-STATE->MAX-STEPS$INLINE
and AGENT-STATE->STEP-COUNTER$INLINE.  REMAINING-STEPS is compliant
with Common Lisp.

Summary
Form:  ( DEFUN REMAINING-STEPS ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION AGENT-STATE->MAX-STEPS$INLINE)
        (:TYPE-PRESCRIPTION AGENT-STATE->STEP-COUNTER$INLINE)
        (:TYPE-PRESCRIPTION NFIX))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM NATP-OF-REMAINING-STEPS

In [17]:
;; Deduct tool cost from budgets
(define deduct-tool-cost ((tool tool-spec-p) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((token-cost (tool-spec->token-cost tool))
       (time-cost (tool-spec->time-cost tool))
       (old-tokens (agent-state->token-budget st))
       (old-time (agent-state->time-budget st)))
    (change-agent-state st
      :token-budget (nfix (- old-tokens token-cost))
      :time-budget (nfix (- old-time time-cost)))))


Since DEDUCT-TOOL-COST is non-recursive, its admission is trivial.
We observe that the type of DEDUCT-TOOL-COST is described by the theorem
(AND (CONSP (DEDUCT-TOOL-COST TOOL ST))
     (TRUE-LISTP (DEDUCT-TOOL-COST TOOL ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for DEDUCT-TOOL-COST....

The non-trivial part of the guard conjecture for DEDUCT-TOOL-COST is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (TOOL-SPEC-P TOOL))
 (AND
  (TOOL-SPEC-P TOOL)
  (LET ((TOKEN-COST (TOOL-SPEC->TOKEN-COST TOOL)))
   (AND
    (TOOL-SPEC-P TOOL)
    (LET ((TIME-COST (TOOL-SPEC->TIME-COST TOOL)))
     (AND
      (AGENT-STATE-P ST)
      (LET ((OLD-TOKENS (AGENT-STATE->TOKEN-BUDGET ST)))
       (AND
        (AGENT-STATE-P ST)
        (LET ((OLD-TIME (AGENT-STATE->TIME-BUDGET ST)))
         (AND
          (ACL2-NUMBERP TOKEN-COST)
          (ACL2-NUMBERP OLD-TOKENS)
          (ACL2-NUMBERP (- TOKEN-COST))
          (ACL2-NUMBERP TIME-COST)
          (ACL2-NUMBER

In [18]:
;; Increment step counter
(define increment-step ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st
    :step-counter (1+ (agent-state->step-counter st))))


Since INCREMENT-STEP is non-recursive, its admission is trivial.  We
observe that the type of INCREMENT-STEP is described by the theorem
(AND (CONSP (INCREMENT-STEP ST)) (TRUE-LISTP (INCREMENT-STEP ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for INCREMENT-STEP....

The non-trivial part of the guard conjecture for INCREMENT-STEP is

Goal
(IMPLIES
 (AGENT-STATE-P ST)
 (AND (AGENT-STATE-P ST)
      (ACL2-NUMBERP (AGENT-STATE->STEP-COUNTER ST))
      (LET ((CHANGE-AGENT-STATE ST)
            (AGENT-STATE->STEP-COUNTER (+ 1 (AGENT-STATE->STEP-COUNTER ST))))
        (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
             (NATP AGENT-STATE->STEP-COUNTER)
             (STRINGP (AGENT-STATE->SYSTEM-PROMPT CHANGE-AGENT-STATE))
             (NATP (AGENT-STATE->FILE-ACCESS CHANGE-AGENT-STATE))
             (NATP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
             (BOOLEANP (AGENT-STATE->DONE CHANGE-AGENT-STATE))
             (CHAT-MESSAGE-LIST-P (

In [19]:
;; Update satisfaction level
(define update-satisfaction ((new-sat natp) (st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st :satisfaction new-sat))


Since UPDATE-SATISFACTION is non-recursive, its admission is trivial.
We observe that the type of UPDATE-SATISFACTION is described by the
theorem 
(AND (CONSP (UPDATE-SATISFACTION NEW-SAT ST))
     (TRUE-LISTP (UPDATE-SATISFACTION NEW-SAT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for UPDATE-SATISFACTION....

The non-trivial part of the guard conjecture for UPDATE-SATISFACTION
is

Goal
(IMPLIES
     (AND (AGENT-STATE-P ST) (NATP NEW-SAT))
     (LET ((CHANGE-AGENT-STATE ST)
           (AGENT-STATE->SATISFACTION NEW-SAT))
       (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
            (NATP (AGENT-STATE->STEP-COUNTER CHANGE-AGENT-STATE))
            (STRINGP (AGENT-STATE->SYSTEM-PROMPT CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->FILE-ACCESS CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
            (BOOLEANP (AGENT-STATE->DONE CHANGE-AGENT-STATE))
            (CHAT-MESSAGE-LIST-P (AGENT-STATE->MESSA

In [20]:
;; Mark agent as done
(define mark-done ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st :done t))


Since MARK-DONE is non-recursive, its admission is trivial.  We observe
that the type of MARK-DONE is described by the theorem 
(AND (CONSP (MARK-DONE ST)) (TRUE-LISTP (MARK-DONE ST))).  We used
the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for MARK-DONE....

The non-trivial part of the guard conjecture for MARK-DONE is

Goal
(IMPLIES
     (AGENT-STATE-P ST)
     (LET ((CHANGE-AGENT-STATE ST)
           (AGENT-STATE->DONE T))
       (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
            (NATP (AGENT-STATE->STEP-COUNTER CHANGE-AGENT-STATE))
            (STRINGP (AGENT-STATE->SYSTEM-PROMPT CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->FILE-ACCESS CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
            (BOOLEANP AGENT-STATE->DONE)
            (CHAT-MESSAGE-LIST-P (AGENT-STATE->MESSAGES CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->MAX-STEPS CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->SATISFACTION CH

In [21]:
;; Set error state
(define set-error ((err error-kind-p) (st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st :error-state err))


Since SET-ERROR is non-recursive, its admission is trivial.  We observe
that the type of SET-ERROR is described by the theorem 
(AND (CONSP (SET-ERROR ERR ST)) (TRUE-LISTP (SET-ERROR ERR ST))). 
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for SET-ERROR....

The non-trivial part of the guard conjecture for SET-ERROR is

Goal
(IMPLIES
     (AND (AGENT-STATE-P ST)
          (ERROR-KIND-P ERR))
     (LET ((CHANGE-AGENT-STATE ST)
           (AGENT-STATE->ERROR-STATE ERR))
       (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
            (NATP (AGENT-STATE->STEP-COUNTER CHANGE-AGENT-STATE))
            (STRINGP (AGENT-STATE->SYSTEM-PROMPT CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->FILE-ACCESS CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
            (BOOLEANP (AGENT-STATE->DONE CHANGE-AGENT-STATE))
            (CHAT-MESSAGE-LIST-P (AGENT-STATE->MESSAGES CHANGE-AGENT-STATE))
            (NATP (AGENT-STATE->MAX-S

In [22]:
;; Initialize conversation with system prompt in agent state
(define init-agent-conversation ((system-prompt stringp) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((max-ctx (agent-state->max-context-tokens st))
       (init-msgs (init-conversation system-prompt max-ctx)))
    (change-agent-state st
      :system-prompt system-prompt
      :messages init-msgs)))


Since INIT-AGENT-CONVERSATION is non-recursive, its admission is trivial.
We observe that the type of INIT-AGENT-CONVERSATION is described by
the theorem 
(AND (CONSP (INIT-AGENT-CONVERSATION SYSTEM-PROMPT ST))
     (TRUE-LISTP (INIT-AGENT-CONVERSATION SYSTEM-PROMPT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for INIT-AGENT-CONVERSATION....

The non-trivial part of the guard conjecture for INIT-AGENT-CONVERSATION
is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (STRINGP SYSTEM-PROMPT))
 (AND
  (AGENT-STATE-P ST)
  (LET ((MAX-CTX (AGENT-STATE->MAX-CONTEXT-TOKENS ST)))
   (AND
    (STRINGP SYSTEM-PROMPT)
    (NATP MAX-CTX)
    (LET ((INIT-MSGS (INIT-CONVERSATION SYSTEM-PROMPT MAX-CTX)))
     (LET ((CHANGE-AGENT-STATE ST)
           (AGENT-STATE->MESSAGES INIT-MSGS)
           (AGENT-STATE->SYSTEM-PROMPT SYSTEM-PROMPT))
      (AND
          (AGENT-STATE-P CHANGE-AGENT-STATE)
          (NATP (AGENT-STATE->STEP-COUNTER CHANGE-AGENT-STATE))
      

In [23]:
;; Add user message to conversation
(define add-user-msg ((content stringp) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((max-ctx (agent-state->max-context-tokens st))
       (old-msgs (agent-state->messages st))
       (new-msgs (add-user-message content old-msgs max-ctx)))
    (change-agent-state st :messages new-msgs)))


Since ADD-USER-MSG is non-recursive, its admission is trivial.  We
observe that the type of ADD-USER-MSG is described by the theorem 
(AND (CONSP (ADD-USER-MSG CONTENT ST))
     (TRUE-LISTP (ADD-USER-MSG CONTENT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for ADD-USER-MSG....

The non-trivial part of the guard conjecture for ADD-USER-MSG is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (STRINGP CONTENT))
 (AND
  (AGENT-STATE-P ST)
  (LET ((MAX-CTX (AGENT-STATE->MAX-CONTEXT-TOKENS ST)))
   (AND
    (AGENT-STATE-P ST)
    (LET ((OLD-MSGS (AGENT-STATE->MESSAGES ST)))
     (AND
      (STRINGP CONTENT)
      (NATP MAX-CTX)
      (CHAT-MESSAGE-LIST-P OLD-MSGS)
      (LET ((NEW-MSGS (ADD-USER-MESSAGE CONTENT OLD-MSGS MAX-CTX)))
       (LET ((CHANGE-AGENT-STATE ST)
             (AGENT-STATE->MESSAGES NEW-MSGS))
        (AND
         (AGENT-STATE-P CHANGE-AGENT-STATE)
         (NATP (AGENT-STATE->STEP-COUNTER CHANGE-AGENT-STATE))
         (STRINGP (A

In [24]:
;; Add assistant message to conversation
(define add-assistant-msg ((content stringp) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((max-ctx (agent-state->max-context-tokens st))
       (old-msgs (agent-state->messages st))
       (new-msgs (add-assistant-message content old-msgs max-ctx)))
    (change-agent-state st :messages new-msgs)))


Since ADD-ASSISTANT-MSG is non-recursive, its admission is trivial.
We observe that the type of ADD-ASSISTANT-MSG is described by the theorem
(AND (CONSP (ADD-ASSISTANT-MSG CONTENT ST))
     (TRUE-LISTP (ADD-ASSISTANT-MSG CONTENT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for ADD-ASSISTANT-MSG....

The non-trivial part of the guard conjecture for ADD-ASSISTANT-MSG
is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (STRINGP CONTENT))
 (AND
  (AGENT-STATE-P ST)
  (LET ((MAX-CTX (AGENT-STATE->MAX-CONTEXT-TOKENS ST)))
   (AND
    (AGENT-STATE-P ST)
    (LET ((OLD-MSGS (AGENT-STATE->MESSAGES ST)))
     (AND
      (STRINGP CONTENT)
      (NATP MAX-CTX)
      (CHAT-MESSAGE-LIST-P OLD-MSGS)
      (LET ((NEW-MSGS (ADD-ASSISTANT-MESSAGE CONTENT OLD-MSGS MAX-CTX)))
       (LET ((CHANGE-AGENT-STATE ST)
             (AGENT-STATE->MESSAGES NEW-MSGS))
        (AND
         (AGENT-STATE-P CHANGE-AGENT-STATE)
         (NATP (AGENT-STATE->STEP-COUNTER CHANGE-A

In [25]:
;; Add tool result to conversation (with output truncation)
(define add-tool-result ((content stringp) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((max-ctx (agent-state->max-context-tokens st))
       (old-msgs (agent-state->messages st))
       ;; Pass t for truncate-output-p to truncate long tool outputs
       (new-msgs (add-tool-message content old-msgs max-ctx t)))
    (change-agent-state st :messages new-msgs)))


Since ADD-TOOL-RESULT is non-recursive, its admission is trivial. 
We observe that the type of ADD-TOOL-RESULT is described by the theorem
(AND (CONSP (ADD-TOOL-RESULT CONTENT ST))
     (TRUE-LISTP (ADD-TOOL-RESULT CONTENT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for ADD-TOOL-RESULT....

The non-trivial part of the guard conjecture for ADD-TOOL-RESULT, given
the :executable-counterpart of BOOLEANP, is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (STRINGP CONTENT))
 (AND
  (AGENT-STATE-P ST)
  (LET ((MAX-CTX (AGENT-STATE->MAX-CONTEXT-TOKENS ST)))
   (AND
    (AGENT-STATE-P ST)
    (LET ((OLD-MSGS (AGENT-STATE->MESSAGES ST)))
     (AND
      (STRINGP CONTENT)
      (NATP MAX-CTX)
      (CHAT-MESSAGE-LIST-P OLD-MSGS)
      (LET ((NEW-MSGS (ADD-TOOL-MESSAGE CONTENT OLD-MSGS MAX-CTX T)))
       (LET ((CHANGE-AGENT-STATE ST)
             (AGENT-STATE->MESSAGES NEW-MSGS))
        (AND
         (AGENT-STATE-P CHANGE-AGENT-STATE)
         (NATP (

In [26]:
;; Get messages for LLM call
(define get-messages ((st agent-state-p))
  :returns (msgs chat-message-list-p)
  (agent-state->messages st))


Since GET-MESSAGES is non-recursive, its admission is trivial.  We
observe that the type of GET-MESSAGES is described by the theorem 
(TRUE-LISTP (GET-MESSAGES ST)).  We used the :type-prescription rule
AGENT-STATE->MESSAGES$INLINE.

Computing the guard conjecture for GET-MESSAGES....

The guard conjecture for GET-MESSAGES is trivial to prove.  GET-MESSAGES
is compliant with Common Lisp.

Summary
Form:  ( DEFUN GET-MESSAGES ...)
Rules: ((:TYPE-PRESCRIPTION AGENT-STATE->MESSAGES$INLINE))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM CHAT-MESSAGE-LIST-P-OF-GET-MESSAGES
          (B* ((MSGS (GET-MESSAGES ST)))
            (CHAT-MESSAGE-LIST-P MSGS))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'GET-MESSAGES
                                                        ID WORLD))
          :RULE-CLASSES :REWRITE)

ACL2 Warning [Non-rec] in ( DEFTHM CHAT-MESSAGE-LIST-P-OF-GET-MESSAGES
...):  A :REWRITE rule generated from CHAT-MESSAGE-LIST-P-OF-GET-MESS

In [27]:
;; Check if conversation has room for more messages
(define conversation-has-room-p ((st agent-state-p))
  :returns (result booleanp)
  (b* ((max-ctx (agent-state->max-context-tokens st))
       (current-msgs (agent-state->messages st)))
    (messages-fit-p current-msgs max-ctx)))


Since CONVERSATION-HAS-ROOM-P is non-recursive, its admission is trivial.
We observe that the type of CONVERSATION-HAS-ROOM-P is described by
the theorem 
(OR (EQUAL (CONVERSATION-HAS-ROOM-P ST) T)
    (EQUAL (CONVERSATION-HAS-ROOM-P ST)
           NIL)).
We used the :type-prescription rule MESSAGES-FIT-P.

Computing the guard conjecture for CONVERSATION-HAS-ROOM-P....

The non-trivial part of the guard conjecture for CONVERSATION-HAS-ROOM-P
is

Goal
(IMPLIES (AGENT-STATE-P ST)
         (AND (AGENT-STATE-P ST)
              (LET ((MAX-CTX (AGENT-STATE->MAX-CONTEXT-TOKENS ST)))
                (AND (AGENT-STATE-P ST)
                     (LET ((CURRENT-MSGS (AGENT-STATE->MESSAGES ST)))
                       (AND (CHAT-MESSAGE-LIST-P CURRENT-MSGS)
                            (NATP MAX-CTX))))))).

Q.E.D.

That completes the proof of the guard theorem for CONVERSATION-HAS-ROOM-P.
CONVERSATION-HAS-ROOM-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN CONVERSATION-HAS-ROOM-P ...)


In [28]:
;; Helper to prove that conversation functions preserve must-respond-p fields
(defthm add-assistant-msg-preserves-done
  (equal (agent-state->done (add-assistant-msg content st))
         (agent-state->done st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-DONE
...):  In a :REWRITE rule generated from ADD-ASSISTANT-MSG-PRESERVES-DONE,
equivalence relation AGENT-STATE-EQUIV$INLINE is maintained at one
problematic occurrence of variable ST in the right-hand side, but not
at any binding occurrence of ST.  Consider replacing that occurrence
of ST in the right-hand side with (DOUBLE-REWRITE ST).  See :doc double-
rewrite for more information on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-DONE ...)
Rules: ((:COMPOUND-RECOGNIZER BOOLEANP-COMPOUND-RECOGNIZER)
        (:DEFINITION ADD-ASSISTANT-MSG)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->DONE-OF-AGENT-STATE)
        (:REWRITE BOOL-FIX-WHEN-BOOLEANP)
        (:TYPE-PRESCRIPTION AGENT-STATE->DONE$INLINE))
Warnings:  Double-rewrite
Time:  0.03 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  138
 ADD-ASSISTANT-MSG-PRESERVES-DONE


In [29]:
(defthm add-assistant-msg-preserves-error-state
  (equal (agent-state->error-state (add-assistant-msg content st))
         (agent-state->error-state st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-ASSISTANT-MSG-PRESERVES-ERROR-STATE ...):  In a :REWRITE rule generated
from ADD-ASSISTANT-MSG-PRESERVES-ERROR-STATE, equivalence relation
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the right-hand side, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the right-hand
side with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-ERROR-STATE ...)
Rules: ((:DEFINITION ADD-ASSISTANT-MSG)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->ERROR-STATE-OF-AGENT-STATE)
        (:REWRITE ERROR-KIND-FIX-WHEN-ERROR-KIND-P)
        (:REWRITE ERROR-KIND-P-OF-AGENT-STATE->ERROR-STATE))
Warnings:  Double-rewrite
Time:  0.03 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  141
 ADD-ASSISTANT-MSG-PRESERVES-ERROR-STATE


In [30]:
(defthm add-assistant-msg-preserves-step-counter
  (equal (agent-state->step-counter (add-assistant-msg content st))
         (agent-state->step-counter st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-ASSISTANT-MSG-PRESERVES-STEP-COUNTER ...):  In a :REWRITE rule
generated from ADD-ASSISTANT-MSG-PRESERVES-STEP-COUNTER, equivalence
relation AGENT-STATE-EQUIV$INLINE is maintained at one problematic
occurrence of variable ST in the right-hand side, but not at any binding
occurrence of ST.  Consider replacing that occurrence of ST in the
right-hand side with (DOUBLE-REWRITE ST).  See :doc double-rewrite
for more information on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-STEP-COUNTER ...)
Rules: ((:DEFINITION ADD-ASSISTANT-MSG)
        (:DEFINITION NFIX)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->STEP-COUNTER-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->STEP-COUNTER$INLINE))
Warnings:  Double-rewrite
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.01)
Prover steps counted:  145
 ADD-ASSISTANT-MSG-PRESERVES-STEP-COUNTER


In [31]:
(defthm add-assistant-msg-preserves-max-steps
  (equal (agent-state->max-steps (add-assistant-msg content st))
         (agent-state->max-steps st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-ASSISTANT-MSG-PRESERVES-MAX-STEPS ...):  In a :REWRITE rule generated
from ADD-ASSISTANT-MSG-PRESERVES-MAX-STEPS, equivalence relation 
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the right-hand side, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the right-hand
side with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-MAX-STEPS ...)
Rules: ((:DEFINITION ADD-ASSISTANT-MSG)
        (:DEFINITION NFIX)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->MAX-STEPS-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->MAX-STEPS$INLINE))
Warnings:  Double-rewrite
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  145
 ADD-ASSISTANT-MSG-PRESERVES-MAX-STEPS


In [32]:
(defthm add-assistant-msg-preserves-token-budget
  (equal (agent-state->token-budget (add-assistant-msg content st))
         (agent-state->token-budget st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-ASSISTANT-MSG-PRESERVES-TOKEN-BUDGET ...):  In a :REWRITE rule
generated from ADD-ASSISTANT-MSG-PRESERVES-TOKEN-BUDGET, equivalence
relation AGENT-STATE-EQUIV$INLINE is maintained at one problematic
occurrence of variable ST in the right-hand side, but not at any binding
occurrence of ST.  Consider replacing that occurrence of ST in the
right-hand side with (DOUBLE-REWRITE ST).  See :doc double-rewrite
for more information on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-TOKEN-BUDGET ...)
Rules: ((:DEFINITION ADD-ASSISTANT-MSG)
        (:DEFINITION NFIX)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->TOKEN-BUDGET-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->TOKEN-BUDGET$INLINE))
Warnings:  Double-rewrite
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  145
 ADD-ASSISTANT-MSG-PRESERVES-TOKEN-BUDGET


In [33]:
(defthm add-assistant-msg-preserves-time-budget
  (equal (agent-state->time-budget (add-assistant-msg content st))
         (agent-state->time-budget st))
  :hints (("Goal" :in-theory (enable add-assistant-msg))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-ASSISTANT-MSG-PRESERVES-TIME-BUDGET ...):  In a :REWRITE rule generated
from ADD-ASSISTANT-MSG-PRESERVES-TIME-BUDGET, equivalence relation
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the right-hand side, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the right-hand
side with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-TIME-BUDGET ...)
Rules: ((:DEFINITION ADD-ASSISTANT-MSG)
        (:DEFINITION NFIX)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->TIME-BUDGET-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->TIME-BUDGET$INLINE))
Warnings:  Double-rewrite
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  145
 ADD-ASSISTANT-MSG-PRESERVES-TIME-BUDGET


In [34]:
;; Derived theorem: add-assistant-msg preserves has-error-p
(defthm add-assistant-msg-preserves-has-error-p
  (equal (has-error-p (add-assistant-msg content st))
         (has-error-p st))
  :hints (("Goal" :in-theory (enable has-error-p))))

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-HAS-ERROR-P ...)
Rules: ((:DEFINITION HAS-ERROR-P)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-ERROR-STATE))
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.01)
Prover steps counted:  81
 ADD-ASSISTANT-MSG-PRESERVES-HAS-ERROR-P


In [35]:
;; Key theorem: add-assistant-msg preserves must-respond-p
(defthm add-assistant-msg-preserves-must-respond-p
  (equal (must-respond-p (add-assistant-msg content st))
         (must-respond-p st))
  :hints (("Goal" :in-theory (enable must-respond-p))))

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-MUST-RESPOND-P ...)
Rules: ((:DEFINITION MUST-RESPOND-P)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-DONE)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-HAS-ERROR-P)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-MAX-STEPS)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-STEP-COUNTER)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-TIME-BUDGET)
        (:REWRITE ADD-ASSISTANT-MSG-PRESERVES-TOKEN-BUDGET))
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.01)
Prover steps counted:  193
 ADD-ASSISTANT-MSG-PRESERVES-MUST-RESPOND-P


In [36]:
;; Same lemmas for add-tool-result
(defthm add-tool-result-preserves-done
  (equal (agent-state->done (add-tool-result content st))
         (agent-state->done st))
  :hints (("Goal" :in-theory (enable add-tool-result))))


ACL2 Warning [Double-rewrite] in ( DEFTHM ADD-TOOL-RESULT-PRESERVES-DONE
...):  In a :REWRITE rule generated from ADD-TOOL-RESULT-PRESERVES-DONE,
equivalence relation AGENT-STATE-EQUIV$INLINE is maintained at one
problematic occurrence of variable ST in the right-hand side, but not
at any binding occurrence of ST.  Consider replacing that occurrence
of ST in the right-hand side with (DOUBLE-REWRITE ST).  See :doc double-
rewrite for more information on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-TOOL-RESULT-PRESERVES-DONE ...)
Rules: ((:COMPOUND-RECOGNIZER BOOLEANP-COMPOUND-RECOGNIZER)
        (:DEFINITION ADD-TOOL-RESULT)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->DONE-OF-AGENT-STATE)
        (:REWRITE BOOL-FIX-WHEN-BOOLEANP)
        (:TYPE-PRESCRIPTION AGENT-STATE->DONE$INLINE))
Warnings:  Double-rewrite
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  141
 ADD-TOOL-RESULT-PRESERVES-DONE


In [37]:
(defthm add-tool-result-preserves-error-state
  (equal (agent-state->error-state (add-tool-result content st))
         (agent-state->error-state st))
  :hints (("Goal" :in-theory (enable add-tool-result))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
ADD-TOOL-RESULT-PRESERVES-ERROR-STATE ...):  In a :REWRITE rule generated
from ADD-TOOL-RESULT-PRESERVES-ERROR-STATE, equivalence relation 
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the right-hand side, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the right-hand
side with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-TOOL-RESULT-PRESERVES-ERROR-STATE ...)
Rules: ((:DEFINITION ADD-TOOL-RESULT)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->ERROR-STATE-OF-AGENT-STATE)
        (:REWRITE ERROR-KIND-FIX-WHEN-ERROR-KIND-P)
        (:REWRITE ERROR-KIND-P-OF-AGENT-STATE->ERROR-STATE))
Warnings:  Double-rewrite
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  144
 ADD-TOOL-RESULT-PRESERVES-ERROR-STATE


In [38]:
(defthm add-tool-result-preserves-has-error-p
  (equal (has-error-p (add-tool-result content st))
         (has-error-p st))
  :hints (("Goal" :in-theory (enable has-error-p))))

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM ADD-TOOL-RESULT-PRESERVES-HAS-ERROR-P ...)
Rules: ((:DEFINITION HAS-ERROR-P)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE ADD-TOOL-RESULT-PRESERVES-ERROR-STATE))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Prover steps counted:  81
 ADD-TOOL-RESULT-PRESERVES-HAS-ERROR-P


In [39]:
;; Abstract external tool calls - models MCP tools (Oxigraph, Qdrant, LLM)
;; ACL2 proves properties GIVEN ANY implementation satisfying these axioms
(encapsulate
  (((external-tool-call * *) => *))
  
  ;; Local witness function
  (local (defun external-tool-call (tool-name query)
           (declare (ignore tool-name query))
           (list 'success nil)))
  
  ;; Axiom: external calls return a list
  (defthm external-tool-call-returns-list
    (true-listp (external-tool-call tool-name query)))
  
  ;; Axiom: response has bounded length (resource safety)
  (defthm external-tool-call-bounded
    (< (len (external-tool-call tool-name query)) 10000)))


To verify that the three encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(LOCAL (DEFUN EXTERNAL-TOOL-CALL (TOOL-NAME QUERY)
                 (DECLARE (IGNORE TOOL-NAME QUERY))
                 (LIST 'SUCCESS NIL)))

Since EXTERNAL-TOOL-CALL is non-recursive, its admission is trivial.
We observe that the type of EXTERNAL-TOOL-CALL is described by the
theorem 
(AND (CONSP (EXTERNAL-TOOL-CALL TOOL-NAME QUERY))
     (TRUE-LISTP (EXTERNAL-TOOL-CALL TOOL-NAME QUERY))).

Summary
Form:  ( DEFUN EXTERNAL-TOOL-CALL ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.00)
EXTERNAL-TOOL-CALL


ACL2 !>>(DEFTHM EXTERNAL-TOOL-CALL-RETURNS-LIST
          (TRUE-LISTP (EXTERNAL-TOOL-CALL TOOL-NAME QUERY)))

ACL2 Warning [Non-rec] in ( DEFTHM EXTERNAL-TOOL-CALL-RETURNS-LIST
...):  A :REWRITE rule generated from EXTERNAL-TOOL-CALL-RETURNS-LIST
will be triggered only by terms c

In [40]:
;; Execute one ReAct step: process action and update state
(define react-step ((action agent-action-p) (tool tool-spec-p) (st agent-state-p))
  :returns (new-st agent-state-p)
  :guard (not (must-respond-p st))
  (b* ((st (increment-step st)))
    (agent-action-case action
      :tool-call
      (if (can-invoke-tool-p tool st)
          (deduct-tool-cost tool st)
        (set-error (error-kind-precondition-failed 
                     "Tool not permitted or insufficient budget")
                   st))
      :final-answer
      (mark-done st)
      :no-action
      st))
  :guard-hints (("Goal" :in-theory (enable must-respond-p))))


Since REACT-STEP is non-recursive, its admission is trivial.  We observe
that the type of REACT-STEP is described by the theorem 
(AND (CONSP (REACT-STEP ACTION TOOL ST))
     (TRUE-LISTP (REACT-STEP ACTION TOOL ST))).
We used the :type-prescription rules DEDUCT-TOOL-COST, INCREMENT-STEP,
MARK-DONE and SET-ERROR.

Computing the guard conjecture for REACT-STEP....

The non-trivial part of the guard conjecture for REACT-STEP, given
the :executable-counterparts of EQLABLEP, ERROR-KIND-P and 
ERROR-KIND-PRECONDITION-FAILED, is

Goal
(IMPLIES (AND (NOT (MUST-RESPOND-P ST))
              (AGENT-STATE-P ST)
              (TOOL-SPEC-P TOOL)
              (AGENT-ACTION-P ACTION))
         (AND (AGENT-STATE-P ST)
              (LET ((ST (INCREMENT-STEP ST)))
                (AND (AGENT-ACTION-P ACTION)
                     (LET ((ACTION.KIND (AGENT-ACTION-KIND ACTION)))
                       (AND (OR (NOT (EQL ACTION.KIND :TOOL-CALL))
                                (TOOL-SPEC-P TOOL))
       

In [41]:
;; Execute ReAct step with conversation management
;; Takes assistant's thought+action and tool result, updates both state and conversation
(define react-step-with-conversation ((action agent-action-p)
                                      (tool tool-spec-p)
                                      (assistant-msg stringp)
                                      (tool-result stringp)
                                      (st agent-state-p))
  :returns (new-st agent-state-p)
  :guard (not (must-respond-p st))
  (b* (;; First add assistant's response to conversation
       (st (add-assistant-msg assistant-msg st))
       ;; Execute the state transition
       (st (react-step action tool st))
       ;; For tool calls, add the tool result if no error
       (st (if (and (agent-action-case action :tool-call)
                    (not (has-error-p st)))
               (add-tool-result tool-result st)
             st)))
    st)
  :guard-hints (("Goal" :in-theory (enable must-respond-p react-step))))


Since REACT-STEP-WITH-CONVERSATION is non-recursive, its admission
is trivial.  We observe that the type of REACT-STEP-WITH-CONVERSATION
is described by the theorem 
(AND (CONSP (REACT-STEP-WITH-CONVERSATION ACTION
                                          TOOL ASSISTANT-MSG TOOL-RESULT ST))
     (TRUE-LISTP
          (REACT-STEP-WITH-CONVERSATION ACTION
                                        TOOL ASSISTANT-MSG TOOL-RESULT ST))).
We used the :type-prescription rules ADD-TOOL-RESULT and REACT-STEP.

Computing the guard conjecture for REACT-STEP-WITH-CONVERSATION....

The non-trivial part of the guard conjecture for REACT-STEP-WITH-CONVERSATION
is

Goal
(IMPLIES (AND (NOT (MUST-RESPOND-P ST))
              (AGENT-STATE-P ST)
              (STRINGP TOOL-RESULT)
              (STRINGP ASSISTANT-MSG)
              (TOOL-SPEC-P TOOL)
              (AGENT-ACTION-P ACTION))
         (AND (STRINGP ASSISTANT-MSG)
              (AGENT-STATE-P ST)
              (LET ((ST (ADD-ASSISTANT-MSG ASSIS

In [42]:
;; Theorem: can-invoke implies tool-permitted (permission safety)
(defthm permission-safety
  (implies (and (tool-spec-p tool)
                (agent-state-p st)
                (can-invoke-tool-p tool st))
           (tool-permitted-p tool st))
  :hints (("Goal" :in-theory (enable can-invoke-tool-p))))


Q.E.D.

The storage of PERMISSION-SAFETY depends upon the :type-prescription
rule TOOL-PERMITTED-P.

Summary
Form:  ( DEFTHM PERMISSION-SAFETY ...)
Rules: ((:DEFINITION CAN-INVOKE-TOOL-P)
        (:TYPE-PRESCRIPTION TOOL-PERMITTED-P))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  39
 PERMISSION-SAFETY


In [43]:
;; Theorem: budgets remain non-negative after deduction
(defthm budget-bounds-after-deduct
  (implies (and (tool-spec-p tool)
                (agent-state-p st))
           (and (natp (agent-state->token-budget (deduct-tool-cost tool st)))
                (natp (agent-state->time-budget (deduct-tool-cost tool st)))))
  :hints (("Goal" :in-theory (enable deduct-tool-cost))))


ACL2 Warning [Non-rec] in ( DEFTHM BUDGET-BOUNDS-AFTER-DEDUCT ...):
A :REWRITE rule generated from BUDGET-BOUNDS-AFTER-DEDUCT will be triggered
only by terms containing the function symbol NATP, which has a non-
recursive definition.  Unless this definition is disabled, this rule
is unlikely ever to be used.


ACL2 Warning [Subsume] in ( DEFTHM BUDGET-BOUNDS-AFTER-DEDUCT ...):
The previously added rule NATP-OF-AGENT-STATE->TOKEN-BUDGET subsumes
a newly proposed :REWRITE rule generated from BUDGET-BOUNDS-AFTER-DEDUCT,
in the sense that the old rule rewrites a more general target.  Because
the new rule will be tried first, it may nonetheless find application.


ACL2 Warning [Non-rec] in ( DEFTHM BUDGET-BOUNDS-AFTER-DEDUCT ...):
A :REWRITE rule generated from BUDGET-BOUNDS-AFTER-DEDUCT will be triggered
only by terms containing the function symbol NATP, which has a non-
recursive definition.  Unless this definition is disabled, this rule
is unlikely ever to be used.


ACL2 Warning [Subsu

In [44]:
;; Theorem: step counter increases after increment
(defthm step-increases-after-increment
  (implies (agent-state-p st)
           (> (agent-state->step-counter (increment-step st))
              (agent-state->step-counter st)))
  :hints (("Goal" :in-theory (enable increment-step))))

Goal'
Goal''

Q.E.D.

Summary
Form:  ( DEFTHM STEP-INCREASES-AFTER-INCREMENT ...)
Rules: ((:DEFINITION INCREMENT-STEP)
        (:DEFINITION NFIX)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->STEP-COUNTER-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->STEP-COUNTER$INLINE))
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  153
 STEP-INCREASES-AFTER-INCREMENT


In [45]:
;; Theorem: reaching max-steps forces must-respond (termination bound)
(defthm termination-by-max-steps
  (implies (and (agent-state-p st)
                (>= (agent-state->step-counter st)
                    (agent-state->max-steps st)))
           (must-respond-p st))
  :hints (("Goal" :in-theory (enable must-respond-p))))


ACL2 Warning [Double-rewrite] in ( DEFTHM TERMINATION-BY-MAX-STEPS
...):  In a :REWRITE rule generated from TERMINATION-BY-MAX-STEPS,
equivalence relation AGENT-STATE-EQUIV$INLINE is maintained at two
problematic occurrences of variable ST in the second hypothesis, but
not at any binding occurrence of ST.  Consider replacing those two
occurrences of ST in the second hypothesis with (DOUBLE-REWRITE ST).
See :doc double-rewrite for more information on this issue.


Q.E.D.

The storage of TERMINATION-BY-MAX-STEPS depends upon the :type-prescription
rule MUST-RESPOND-P.

Summary
Form:  ( DEFTHM TERMINATION-BY-MAX-STEPS ...)
Rules: ((:DEFINITION MUST-RESPOND-P)
        (:DEFINITION NOT)
        (:TYPE-PRESCRIPTION MUST-RESPOND-P))
Warnings:  Double-rewrite
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Prover steps counted:  78
 TERMINATION-BY-MAX-STEPS


In [46]:
;; Theorem: agent state partitions into continue/respond/satisfied
(defthm continue-respond-partition
  (implies (agent-state-p st)
           (or (must-respond-p st)
               (should-continue-p st)
               (>= (agent-state->satisfaction st) *satisfaction-threshold*)))
  :hints (("Goal" :in-theory (enable must-respond-p should-continue-p)))
  :rule-classes nil)


Q.E.D.

Summary
Form:  ( DEFTHM CONTINUE-RESPOND-PARTITION ...)
Rules: ((:DEFINITION MUST-RESPOND-P)
        (:DEFINITION SHOULD-CONTINUE-P)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  156
 CONTINUE-RESPOND-PARTITION


In [47]:
;; Theorem: error state forces must-respond
(defthm error-state-forces-must-respond
  (implies (and (agent-state-p st)
                (has-error-p st))
           (must-respond-p st))
  :hints (("Goal" :in-theory (enable must-respond-p has-error-p))))


Q.E.D.

The storage of ERROR-STATE-FORCES-MUST-RESPOND depends upon the :type-
prescription rule MUST-RESPOND-P.

Summary
Form:  ( DEFTHM ERROR-STATE-FORCES-MUST-RESPOND ...)
Rules: ((:DEFINITION HAS-ERROR-P)
        (:DEFINITION MUST-RESPOND-P)
        (:TYPE-PRESCRIPTION MUST-RESPOND-P))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  113
 ERROR-STATE-FORCES-MUST-RESPOND


In [48]:
;; Theorem: remaining steps decreases (for termination proofs)
(defthm remaining-steps-decreases-after-increment
  (implies (and (agent-state-p st)
                (< (agent-state->step-counter st)
                   (agent-state->max-steps st)))
           (< (remaining-steps (increment-step st))
              (remaining-steps st)))
  :hints (("Goal" :in-theory (enable remaining-steps increment-step))))


ACL2 Warning [Double-rewrite] in ( DEFTHM 
REMAINING-STEPS-DECREASES-AFTER-INCREMENT ...):  In a :REWRITE rule
generated from REMAINING-STEPS-DECREASES-AFTER-INCREMENT, equivalence
relation AGENT-STATE-EQUIV$INLINE is maintained at two problematic
occurrences of variable ST in the second hypothesis, but not at any
binding occurrence of ST.  Consider replacing those two occurrences
of ST in the second hypothesis with (DOUBLE-REWRITE ST).  See :doc
double-rewrite for more information on this issue.

Goal'

Splitter note (see :DOC splitter) for Goal' (3 subgoals).
  if-intro: ((:DEFINITION NFIX))

Subgoal 3
Subgoal 2
Subgoal 1

Q.E.D.

Summary
Form:  ( DEFTHM REMAINING-STEPS-DECREASES-AFTER-INCREMENT ...)
Rules: ((:DEFINITION INCREMENT-STEP)
        (:DEFINITION NFIX)
        (:DEFINITION NOT)
        (:DEFINITION REMAINING-STEPS)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:EXECUTABLE-COUNTERPART UNARY--)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->MAX

In [49]:
(defthm deduct-preserves-agent-state
  (implies (and (tool-spec-p tool) (agent-state-p st))
           (agent-state-p (deduct-tool-cost tool st))))


ACL2 Warning [Subsume] in ( DEFTHM DEDUCT-PRESERVES-AGENT-STATE ...):
The previously added rule AGENT-STATE-P-OF-DEDUCT-TOOL-COST subsumes
a newly proposed :REWRITE rule generated from DEDUCT-PRESERVES-AGENT-STATE,
in the sense that the old rule rewrites a more general target.  Because
the new rule will be tried first, it may nonetheless find application.


Q.E.D.

The storage of DEDUCT-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM DEDUCT-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-DEDUCT-TOOL-COST)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 DEDUCT-PRESERVES-AGENT-STATE


In [50]:
(defthm increment-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (increment-step st))))


ACL2 Warning [Subsume] in ( DEFTHM INCREMENT-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-INCREMENT-STEP subsumes
a newly proposed :REWRITE rule generated from 
INCREMENT-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of INCREMENT-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM INCREMENT-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-INCREMENT-STEP)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  11
 INCREMENT-PRESERVES-AGENT-STATE


In [51]:
(defthm update-satisfaction-preserves-agent-state
  (implies (and (natp new-sat) (agent-state-p st))
           (agent-state-p (update-satisfaction new-sat st))))


ACL2 Warning [Subsume] in ( DEFTHM UPDATE-SATISFACTION-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-UPDATE-SATISFACTION
subsumes a newly proposed :REWRITE rule generated from 
UPDATE-SATISFACTION-PRESERVES-AGENT-STATE, in the sense that the old
rule rewrites a more general target.  Because the new rule will be
tried first, it may nonetheless find application.


Q.E.D.

The storage of UPDATE-SATISFACTION-PRESERVES-AGENT-STATE depends upon
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-
prescription rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM UPDATE-SATISFACTION-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-UPDATE-SATISFACTION)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 UPDATE-SATISFACTION-PRESERVES-AGENT-STATE


In [52]:
(defthm mark-done-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (mark-done st))))


ACL2 Warning [Subsume] in ( DEFTHM MARK-DONE-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-MARK-DONE subsumes
a newly proposed :REWRITE rule generated from 
MARK-DONE-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of MARK-DONE-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM MARK-DONE-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-MARK-DONE)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  11
 MARK-DONE-PRESERVES-AGENT-STATE


In [53]:
(defthm set-error-preserves-agent-state
  (implies (and (error-kind-p err) (agent-state-p st))
           (agent-state-p (set-error err st))))


ACL2 Warning [Subsume] in ( DEFTHM SET-ERROR-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-SET-ERROR subsumes
a newly proposed :REWRITE rule generated from 
SET-ERROR-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of SET-ERROR-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM SET-ERROR-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-SET-ERROR)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 SET-ERROR-PRESERVES-AGENT-STATE


In [54]:
(defthm react-step-preserves-agent-state
  (implies (and (agent-action-p action)
                (tool-spec-p tool)
                (agent-state-p st))
           (agent-state-p (react-step action tool st))))
;; Conversation state transition preservations


ACL2 Warning [Subsume] in ( DEFTHM REACT-STEP-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-REACT-STEP subsumes
a newly proposed :REWRITE rule generated from 
REACT-STEP-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of REACT-STEP-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM REACT-STEP-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-REACT-STEP)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  21
 REACT-STEP-PRESERVES-AGENT-STATE


In [55]:
(defthm init-agent-conversation-preserves-agent-state
  (implies (and (stringp system-prompt) (agent-state-p st))
           (agent-state-p (init-agent-conversation system-prompt st))))


ACL2 Warning [Subsume] in ( DEFTHM 
INIT-AGENT-CONVERSATION-PRESERVES-AGENT-STATE ...):  The previously
added rule AGENT-STATE-P-OF-INIT-AGENT-CONVERSATION subsumes a newly
proposed :REWRITE rule generated from 
INIT-AGENT-CONVERSATION-PRESERVES-AGENT-STATE, in the sense that the
old rule rewrites a more general target.  Because the new rule will
be tried first, it may nonetheless find application.


Q.E.D.

The storage of INIT-AGENT-CONVERSATION-PRESERVES-AGENT-STATE depends
upon the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the
:type-prescription rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM INIT-AGENT-CONVERSATION-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-INIT-AGENT-CONVERSATION)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 INIT-AGENT-CONVERSATION-PRESERVE

In [56]:
(defthm add-user-msg-preserves-agent-state
  (implies (and (stringp content) (agent-state-p st))
           (agent-state-p (add-user-msg content st))))


ACL2 Warning [Subsume] in ( DEFTHM ADD-USER-MSG-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-ADD-USER-MSG subsumes
a newly proposed :REWRITE rule generated from 
ADD-USER-MSG-PRESERVES-AGENT-STATE, in the sense that the old rule
rewrites a more general target.  Because the new rule will be tried
first, it may nonetheless find application.


Q.E.D.

The storage of ADD-USER-MSG-PRESERVES-AGENT-STATE depends upon the
:compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM ADD-USER-MSG-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-ADD-USER-MSG)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 ADD-USER-MSG-PRESERVES-AGENT-STATE


In [57]:
(defthm add-assistant-msg-preserves-agent-state
  (implies (and (stringp content) (agent-state-p st))
           (agent-state-p (add-assistant-msg content st))))


ACL2 Warning [Subsume] in ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-ADD-ASSISTANT-MSG
subsumes a newly proposed :REWRITE rule generated from 
ADD-ASSISTANT-MSG-PRESERVES-AGENT-STATE, in the sense that the old
rule rewrites a more general target.  Because the new rule will be
tried first, it may nonetheless find application.


Q.E.D.

The storage of ADD-ASSISTANT-MSG-PRESERVES-AGENT-STATE depends upon
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-
prescription rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM ADD-ASSISTANT-MSG-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-ADD-ASSISTANT-MSG)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 ADD-ASSISTANT-MSG-PRESERVES-AGENT-STATE


In [58]:
(defthm add-tool-result-preserves-agent-state
  (implies (and (stringp content) (agent-state-p st))
           (agent-state-p (add-tool-result content st))))


ACL2 Warning [Subsume] in ( DEFTHM ADD-TOOL-RESULT-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-ADD-TOOL-RESULT subsumes
a newly proposed :REWRITE rule generated from 
ADD-TOOL-RESULT-PRESERVES-AGENT-STATE, in the sense that the old rule
rewrites a more general target.  Because the new rule will be tried
first, it may nonetheless find application.


Q.E.D.

The storage of ADD-TOOL-RESULT-PRESERVES-AGENT-STATE depends upon the
:compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM ADD-TOOL-RESULT-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-ADD-TOOL-RESULT)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  16
 ADD-TOOL-RESULT-PRESERVES-AGENT-STATE


In [59]:
(defthm react-step-with-conversation-preserves-agent-state
  (implies (and (agent-action-p action)
                (tool-spec-p tool)
                (stringp assistant-msg)
                (stringp tool-result)
                (agent-state-p st))
           (agent-state-p (react-step-with-conversation 
                           action tool assistant-msg tool-result st))))


ACL2 Warning [Subsume] in ( DEFTHM 
REACT-STEP-WITH-CONVERSATION-PRESERVES-AGENT-STATE ...):  The previously
added rule AGENT-STATE-P-OF-REACT-STEP-WITH-CONVERSATION subsumes a
newly proposed :REWRITE rule generated from 
REACT-STEP-WITH-CONVERSATION-PRESERVES-AGENT-STATE, in the sense that
the old rule rewrites a more general target.  Because the new rule
will be tried first, it may nonetheless find application.


Q.E.D.

The storage of REACT-STEP-WITH-CONVERSATION-PRESERVES-AGENT-STATE depends
upon the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the
:type-prescription rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM REACT-STEP-WITH-CONVERSATION-PRESERVES-AGENT-STATE
...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:EXECUTABLE-COUNTERPART IF)
        (:REWRITE AGENT-STATE-P-OF-REACT-STEP-WITH-CONVERSATION)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  31
 RE